질문 - 대답
I am Tom. - You are Jane.
Hello~ - Nice to meet you.
Good morning - Who are you?

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
text = '''I am Tom. - You are Jane.
Hello~ - Nice to meet you.
Good morning - Who are you?'''

In [8]:
lines = text.split('\n')

In [9]:
len(lines)

3

In [10]:
input_texts = []
output_texts = []

input_chars = set()
output_chars = set()

for line in lines:
    s1, s2 = line.split('-')
    s2 = '\t' + s2 + '\n'
    input_texts.append(s1)
    output_texts.append(s2)
    
    for c1 in s1:
        if c1 not in input_chars : input_chars.add(c1)
            
    for c2 in s2:
        if c2 not in output_chars : output_chars.add(c2)
            
            
input_chars = sorted(list(input_chars))
output_chars = sorted(list(output_chars))

In [11]:
print(input_chars)
print(output_chars)

[' ', '.', 'G', 'H', 'I', 'T', 'a', 'd', 'e', 'g', 'i', 'l', 'm', 'n', 'o', 'r', '~']
['\t', '\n', ' ', '.', '?', 'J', 'N', 'W', 'Y', 'a', 'c', 'e', 'h', 'i', 'm', 'n', 'o', 'r', 't', 'u', 'y']


In [13]:
max_input_len = max([len(s) for s in input_texts])
max_output_len = max([len(s) for s in output_texts])

In [14]:
max_input_len, max_output_len

(13, 20)

In [15]:
enc_in_data = np.zeros([3, max_input_len, len(input_chars)])    # (10000,16,70)
dec_in_data = np.zeros([3, max_output_len, len(output_chars)])  # (10000,59,93)
dec_out_data = np.zeros([3, max_output_len, len(output_chars)]) # (10000,59,93)

In [18]:
enc_in_data.shape, dec_in_data.shape,  dec_out_data.shape

((3, 13, 17), (3, 20, 21), (3, 20, 21))

In [19]:
for i, (s1, s2) in enumerate(zip(input_texts, output_texts)):
    for j, c1 in enumerate(s1):
        enc_in_data[i, j, input_chars.index(c1)] = 1
        
    for j, c2 in enumerate(s2):
        dec_in_data[i, j, output_chars.index(c2)] = 1
        if j>0:
            dec_out_data[i, j-1, output_chars.index(c2)] = 1

In [20]:
enc_in_data.shape, dec_in_data.shape, dec_out_data.shape

((3, 13, 17), (3, 20, 21), (3, 20, 21))

In [23]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense


In [24]:
enc_input = Input(shape = (None, 17))

enc_lstm = LSTM(32, return_state = True)
enc_output, enc_h, enc_c = enc_lstm(enc_input)

enc_states = [enc_h, enc_c]


W0828 09:23:39.603420  1480 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\deeplearning\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0828 09:23:39.617421  1480 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\deeplearning\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0828 09:23:39.620421  1480 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\deeplearning\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [31]:
dec_input = Input(shape = (None, 21))
dec_lstm = LSTM(32, return_sequences= True, return_state = True)
dec_outputs,_,_ = dec_lstm(dec_input, initial_state = enc_states)

dec_dense = Dense(21, activation = 'softmax')
dec_outputs = dec_dense(dec_outputs)

In [32]:
model = Model([enc_input, dec_input], dec_outputs)

In [33]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 17)     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None, 21)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 32), (None,  6400        input_1[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 32), ( 6912        input_3[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [34]:
model.compile(loss ='categorical_crossentropy', optimizer = 'rmsprop')

In [36]:
model.fit([enc_in_data, dec_in_data], dec_out_data,
         batch_size =1, epochs = 1000)

Epoch 1/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.1495
Epoch 2/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.1575
Epoch 3/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.1337
Epoch 4/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.1453
Epoch 5/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.1208
Epoch 6/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.1269
Epoch 7/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.1313
Epoch 8/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.1001
Epoch 9/1000
3/3 [==============================] - 0s 12ms/step - loss: 1.1123
Epoch 10/1000
3/3 [==============================] - 0s 12ms/step - loss: 1.1147
Epoch 11/1000
3/3 [==============================] - 0s 12ms/step - loss: 1.0884
Epoch 12/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.1043
Epoch 13/1000
3/3 [==================

3/3 [==============================] - 0s 12ms/step - loss: 0.4812
Epoch 103/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.4918
Epoch 104/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.4670
Epoch 105/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.4644
Epoch 106/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.4780
Epoch 107/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.4474
Epoch 108/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.4416
Epoch 109/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.4324
Epoch 110/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.4278
Epoch 111/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.4226
Epoch 112/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.4178
Epoch 113/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.4254
Epoch 114/1000
3/3 [===========

3/3 [==============================] - 0s 12ms/step - loss: 0.1096
Epoch 203/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.1177
Epoch 204/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0983
Epoch 205/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0947
Epoch 206/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0926
Epoch 207/1000
3/3 [==============================] - 0s 12ms/step - loss: 0.0911
Epoch 208/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0896
Epoch 209/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0879
Epoch 210/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0872
Epoch 211/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0866
Epoch 212/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0875
Epoch 213/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0854
Epoch 214/1000
3/3 [===========

3/3 [==============================] - 0s 11ms/step - loss: 0.0129
Epoch 303/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0125
Epoch 304/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0121
Epoch 305/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0118
Epoch 306/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0113
Epoch 307/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0110
Epoch 308/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0108
Epoch 309/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0106
Epoch 310/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0104
Epoch 311/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0102
Epoch 312/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0110
Epoch 313/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0121
Epoch 314/1000
3/3 [===========

3/3 [==============================] - 0s 11ms/step - loss: 0.0012
Epoch 403/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0012
Epoch 404/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0012
Epoch 405/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0012
Epoch 406/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0011
Epoch 407/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0011
Epoch 408/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0010
Epoch 409/1000
3/3 [==============================] - 0s 11ms/step - loss: 9.8632e-04
Epoch 410/1000
3/3 [==============================] - 0s 11ms/step - loss: 9.9856e-04
Epoch 411/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0011
Epoch 412/1000
3/3 [==============================] - 0s 11ms/step - loss: 0.0010
Epoch 413/1000
3/3 [==============================] - 0s 12ms/step - loss: 9.3407e-04
Epoch 414/1000
3/3 

...

3/3 [==============================] - 0s 11ms/step - loss: 9.5021e-06
Epoch 593/1000
3/3 [==============================] - 0s 11ms/step - loss: 9.3213e-06
Epoch 594/1000
3/3 [==============================] - 0s 11ms/step - loss: 9.1047e-06
Epoch 595/1000
3/3 [==============================] - 0s 11ms/step - loss: 8.9785e-06
Epoch 596/1000
3/3 [==============================] - 0s 11ms/step - loss: 8.8891e-06
Epoch 597/1000
3/3 [==============================] - 0s 11ms/step - loss: 8.7461e-06
Epoch 598/1000
3/3 [==============================] - 0s 12ms/step - loss: 8.4401e-06
Epoch 599/1000
3/3 [==============================] - 0s 11ms/step - loss: 8.2613e-06
Epoch 600/1000
3/3 [==============================] - 0s 11ms/step - loss: 7.9394e-06
Epoch 601/1000
3/3 [==============================] - 0s 12ms/step - loss: 7.7636e-06
Epoch 602/1000
3/3 [==============================] - 0s 10ms/step - loss: 7.5401e-06
Epoch 603/1000
3/3 [==============================] - 0s 11ms/step - 

3/3 [==============================] - 0s 11ms/step - loss: 1.6123e-06
Epoch 688/1000
3/3 [==============================] - 0s 12ms/step - loss: 1.5795e-06
Epoch 689/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.5746e-06
Epoch 690/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.5408e-06
Epoch 691/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.5259e-06
Epoch 692/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.4901e-06
Epoch 693/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.4673e-06
Epoch 694/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.4444e-06
Epoch 695/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.4335e-06
Epoch 696/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.4136e-06
Epoch 697/1000
3/3 [==============================] - 0s 11ms/step - loss: 1.3928e-06
Epoch 698/1000
3/3 [==============================] - 0s 12ms/step - 

3/3 [==============================] - 0s 12ms/step - loss: 5.9803e-07
Epoch 783/1000
3/3 [==============================] - 0s 12ms/step - loss: 5.9108e-07
Epoch 784/1000
3/3 [==============================] - 0s 11ms/step - loss: 5.8512e-07
Epoch 785/1000
3/3 [==============================] - 0s 11ms/step - loss: 5.8015e-07
Epoch 786/1000
3/3 [==============================] - 0s 11ms/step - loss: 5.7022e-07
Epoch 787/1000
3/3 [==============================] - 0s 11ms/step - loss: 5.7618e-07
Epoch 788/1000
3/3 [==============================] - 0s 11ms/step - loss: 5.6624e-07
Epoch 789/1000
3/3 [==============================] - 0s 11ms/step - loss: 5.5929e-07
Epoch 790/1000
3/3 [==============================] - 0s 11ms/step - loss: 5.5730e-07
Epoch 791/1000
3/3 [==============================] - 0s 10ms/step - loss: 5.5134e-07
Epoch 792/1000
3/3 [==============================] - 0s 11ms/step - loss: 5.4836e-07
Epoch 793/1000
3/3 [==============================] - 0s 12ms/step - 

3/3 [==============================] - 0s 12ms/step - loss: 3.3279e-07
Epoch 878/1000
3/3 [==============================] - 0s 11ms/step - loss: 3.3279e-07
Epoch 879/1000
3/3 [==============================] - 0s 11ms/step - loss: 3.2683e-07
Epoch 880/1000
3/3 [==============================] - 0s 11ms/step - loss: 3.2584e-07
Epoch 881/1000
3/3 [==============================] - 0s 11ms/step - loss: 3.2485e-07
Epoch 882/1000
3/3 [==============================] - 0s 11ms/step - loss: 3.2187e-07
Epoch 883/1000
3/3 [==============================] - 0s 11ms/step - loss: 3.2087e-07
Epoch 884/1000
3/3 [==============================] - 0s 11ms/step - loss: 3.1392e-07
Epoch 885/1000
3/3 [==============================] - 0s 12ms/step - loss: 3.1392e-07
Epoch 886/1000
3/3 [==============================] - 0s 11ms/step - loss: 3.1491e-07
Epoch 887/1000
3/3 [==============================] - 0s 11ms/step - loss: 3.0895e-07
Epoch 888/1000
3/3 [==============================] - 0s 11ms/step - 

3/3 [==============================] - 0s 12ms/step - loss: 2.1358e-07
Epoch 973/1000
3/3 [==============================] - 0s 14ms/step - loss: 2.1259e-07
Epoch 974/1000
3/3 [==============================] - 0s 12ms/step - loss: 2.1358e-07
Epoch 975/1000
3/3 [==============================] - 0s 12ms/step - loss: 2.1259e-07
Epoch 976/1000
3/3 [==============================] - 0s 15ms/step - loss: 2.1259e-07
Epoch 977/1000
3/3 [==============================] - 0s 14ms/step - loss: 2.1259e-07
Epoch 978/1000
3/3 [==============================] - 0s 15ms/step - loss: 2.1160e-07
Epoch 979/1000
3/3 [==============================] - 0s 12ms/step - loss: 2.1160e-07
Epoch 980/1000
3/3 [==============================] - 0s 14ms/step - loss: 2.1160e-07
Epoch 981/1000
3/3 [==============================] - 0s 11ms/step - loss: 2.1060e-07
Epoch 982/1000
3/3 [==============================] - 0s 11ms/step - loss: 2.1160e-07
Epoch 983/1000
3/3 [==============================] - 0s 11ms/step - 